In [2]:
import os
import shutil


import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
import datetime
import math
import warnings

import numpy as np
import pandas as pd
import prophet
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from tensorflow.keras.models import Sequential


warnings.filterwarnings("ignore")

In [3]:
#Pour mes datas, j'importe, je choisis le dataset avec les revenus et la date d'ConnectionAbortedError
file = 'tgdata.csv'
df= pd.read_csv(file)
df.head(5)

,Unnamed: 0,dt,subs,revenu_data_pack,BData_1K_700M_subs,BData_200_120M_subs,BData_200_90M_subs,BData_350_170M_subs,BData_350_220M_subs,BData_5000_4G_subs,...,WelcomePack_revenus,XTRANET100_revenus,XTRANET1000_revenus,XTRANET1500_revenus,XTRANET200_revenus,XTRANET3000_revenus,XTRANET350_revenus,XTRANET50_revenus,XTRANET600_revenus,XTRANuit_revenus
0,0,2022-01-01,440,166850.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2022-01-02,370,134750.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,2022-01-03,368,129300.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,2022-01-04,351,135850.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,2022-01-05,361,134350.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df = df.drop('Unnamed: 0', axis=1)
df.head(5)

,dt,subs,revenu_data_pack,BData_1K_700M_subs,BData_200_120M_subs,BData_200_90M_subs,BData_350_170M_subs,BData_350_220M_subs,BData_5000_4G_subs,BData_600_500M_subs,...,WelcomePack_revenus,XTRANET100_revenus,XTRANET1000_revenus,XTRANET1500_revenus,XTRANET200_revenus,XTRANET3000_revenus,XTRANET350_revenus,XTRANET50_revenus,XTRANET600_revenus,XTRANuit_revenus
0,2022-01-01,440,166850.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2022-01-02,370,134750.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2022-01-03,368,129300.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2022-01-04,351,135850.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2022-01-05,361,134350.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
data_time = df[['dt', 'revenu_data_pack']]
data_df = df[['revenu_data_pack']]
print(data_df.shape)
data_df.head(5)


data = data_df.iloc[:800]

(1002, 1)


In [6]:
data = np.array(data)
data

array([[166850.],
       [134750.],
       [129300.],
       [135850.],
       [134350.],
       [125100.],
       [152050.],
       [126150.],
       [146800.],
       [147900.],
       [123450.],
       [118050.],
       [138350.],
       [126150.],
       [139100.],
       [122350.],
       [116250.],
       [126950.],
       [120900.],
       [156650.],
       [109650.],
       [139350.],
       [180800.],
       [117550.],
       [116050.],
       [116050.],
       [116700.],
       [163300.],
       [112500.],
       [140750.],
       [107750.],
       [106600.],
       [133500.],
       [117450.],
       [117250.],
       [101850.],
       [127400.],
       [111000.],
       [147300.],
       [135150.],
       [119250.],
       [122100.],
       [135650.],
       [126450.],
       [154850.],
       [105150.],
       [131000.],
       [118850.],
       [121650.],
       [139600.],
       [134800.],
       [142400.],
       [165050.],
       [122400.],
       [110750.],
       [13

In [7]:
# one-step naive forecast
def naive_forecast(history, n):
    return history[-n]

In [8]:
# test naive forecast
for i in range(1, len(data)+1):
    print(naive_forecast(data, i))

[127043.]
[113148.]
[126747.]
[130549.]
[121744.]
[145694.]
[116746.]
[202496.]
[170646.]
[150844.]
[134395.]
[134296.]
[144338.]
[179291.]
[121244.]
[130393.]
[121994.]
[110395.]
[143140.]
[120842.]
[133545.]
[122943.]
[125895.]
[154694.]
[115046.]
[125095.]
[127997.]
[127695.]
[109197.]
[145498.]
[164193.]
[123695.]
[143791.]
[119545.]
[137046.]
[146495.]
[187595.]
[118595.]
[152296.]
[117943.]
[189046.]
[122645.]
[214745.]
[150746.]
[130895.]
[115999.]
[124998.]
[118393.]
[136097.]
[130496.]
[89948.]
[115595.]
[111446.]
[144995.]
[116393.]
[134092.]
[106394.]
[123545.]
[134944.]
[152446.]
[133342.]
[134346.]
[110945.]
[128795.]
[134845.]
[163998.]
[144492.]
[115498.]
[124145.]
[147247.]
[149445.]
[156545.]
[186945.]
[157142.]
[115744.]
[153848.]
[165647.]
[122946.]
[125397.]
[144347.]
[131394.]
[136696.]
[129245.]
[106999.]
[114998.]
[126897.]
[132449.]
[121199.]
[140149.]
[116800.]
[120700.]
[110250.]
[115000.]
[134950.]
[142500.]
[174700.]
[134150.]
[131250.]
[138150.]
[136400.]
[

In [9]:
history[-n] = naive_forecast(data, i)

In [10]:
#We can now look at developing a function for the average forecast strategy. Averaging the
#last n observations is straight-forward; for example:
# mean forecast
from numpy import mean
result_mean = mean(naive_forecast(data, i))
result_mean

166850.0

In [11]:
# median forecast
from numpy import median
result_median = median(naive_forecast(data, i))
result_median

166850.0

In [12]:
# grid search simple forecasts
from math import sqrt
from numpy import mean
from numpy import median
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from sklearn.metrics import mean_squared_error

# one-step simple forecast
def simple_forecast(history, config):
    n, offset, avg_type = config
    
    # persist value, ignore other config
    if avg_type == 'persist':
        return history[-n]

    # collect values to average
    values = list()
    if offset == 1:
        values = history[-n:]
    else:
        # skip bad configs
        if n*offset > len(history):
            raise Exception('Config beyond end of data: %d %d' % (n,offset))

        # try and collect n values using offset
        for i in range(1, n+1):
            ix = i * offset
            values.append(history[-ix])
            
    # check if we can average
    if len(values) < 2:
        raise Exception('Cannot calculate average')
    # mean of last n values
    if avg_type == 'mean':
        return mean(values)
    # median of last n values
    return median(values)

# root mean squared error or rmse
def measure_rmse(actual, predicted):
    return sqrt(mean_squared_error(actual, predicted))
# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
    return data[:-n_test], data[-n_test:]


In [13]:
# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
    predictions = list()
    # split dataset
    train, test = train_test_split(data, n_test)
    # seed history with training dataset
    history = [x for x in train]
    # step over each time-step in the test set
    for i in range(len(test)):
        # fit model and make forecast for history
        yhat = simple_forecast(history, cfg)
        # store forecast in list of predictions
        predictions.append(yhat)
        # add actual observation to history for the next loop
        history.append(test[i])
    # estimate prediction error
    error = measure_rmse(test, predictions)
    return error



In [14]:

# score a model, return None on failure
def score_model(data, n_test, cfg, debug=False):
    result = None
    # convert config to a key
    key = str(cfg)
    # show all warnings and fail on exception if debugging
    if debug:
        result = walk_forward_validation(data, n_test, cfg)
    else:
        # one failure during model validation suggests an unstable config
        try:
            # never show warnings when grid searching, too noisy
            with catch_warnings():
                filterwarnings("ignore")
            result = walk_forward_validation(data, n_test, cfg)
        except:
            error = None
    # check for an interesting result
    if result is not None:
        print(' > Model[%s] %.3f' % (key, result))
    return (key, result)

In [15]:
# grid search configs
def grid_search(data, cfg_list, n_test, parallel=True):
    scores = None
    if parallel:
        # execute configs in parallel
        executor = Parallel(n_jobs=cpu_count(), backend='multiprocessing')
        tasks = (delayed(score_model)(data, n_test, cfg) for cfg in cfg_list)
        scores = executor(tasks)
    else:
        scores = [score_model(data, n_test, cfg) for cfg in cfg_list]
    # remove empty results
    scores = [r for r in scores if r[1] != None]
    # sort configs by error, asc
    scores.sort(key=lambda tup: tup[1])
    
    return scores

In [ ]:
# create a set of simple configs to try
def simple_configs(max_length, offsets=[1]):
    configs = list()
    for i in range(1, max_length+1):
        for o in offsets:
            for t in ['persist', 'mean', 'median']:
                cfg = [i, o, t]
                configs.append(cfg)
    return configs


if __name__ == '__main__':
# define dataset(here data_df caause it is the whole before split train test)

    # data split
    n_test = 200
    # model configs
    max_length = len(data)- n_test
    cfg_list = simple_configs(max_length)
    # grid search
    scores = grid_search(data, cfg_list, n_test)
    print('done')
    # list top 3 configs
    for cfg, error in scores[:3]:
        print(cfg, error)

#### Okay ici on va relancer le modèle avec les bons hyperparamètres, et prédire tout le test set. Avec les prédicitons et les valeurs actuelles. J'aimerais enregistrer le modèle fitté. 

## Ici nous allons utiliser SARIMA 

### Ici lstm

In [ ]:
# grid search lstm for monthly airline passengers dataset
from math import sqrt
from numpy import array
from numpy import mean
from pandas import DataFrame
from pandas import concat
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
return data[:-n_test], data[-n_test:]
# transform list into supervised learning format
def series_to_supervised(data, n_in, n_out=1):
df = DataFrame(data)
cols = list()
# input sequence (t-n, ... t-1)
for i in range(n_in, 0,-1):
cols.append(df.shift(i))
# forecast sequence (t, t+1, ... t+n)
for i in range(0, n_out):
cols.append(df.shift(-i))
# put it all together
agg = concat(cols, axis=1)
# drop rows with NaN values
agg.dropna(inplace=True)
return agg.values

In [ ]:
# root mean squared error or rmse
def measure_rmse(actual, predicted):
return sqrt(mean_squared_error(actual, predicted))
# difference dataset
def difference(data, order):
return [data[i]- data[i- order] for i in range(order, len(data))]

In [ ]:
# fit a model
def model_fit(train, config):
# unpack config
n_input, n_nodes, n_epochs, n_batch, n_diff = config
# prepare data
if n_diff > 0:
train = difference(train, n_diff)
# transform series into supervised format
data = series_to_supervised(train, n_in=n_input)
# separate inputs and outputs
train_x, train_y = data[:, :-1], data[:,-1]
# reshape input data into [samples, timesteps, features]
n_features = 1
train_x = train_x.reshape((train_x.shape[0], train_x.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(n_nodes, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(n_nodes, activation='relu'))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
# fit model
model.fit(train_x, train_y, epochs=n_epochs, batch_size=n_batch, verbose=0)
return model

In [ ]:
# forecast with the fit model
def model_predict(model, history, config):
# unpack config
n_input, _, _, _, n_diff = config
# prepare data
correction = 0.0
if n_diff > 0:
correction = history[-n_diff]
history = difference(history, n_diff)
# reshape sample into [samples, timesteps, features]
x_input = array(history[-n_input:]).reshape((1, n_input, 1))
# forecast
yhat = model.predict(x_input, verbose=0)
return correction + yhat[0]

In [ ]:
# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
predictions = list()
# split dataset
train, test = train_test_split(data, n_test)
# fit model
model = model_fit(train, cfg)
# seed history with training dataset
history = [x for x in train]
# step over each time-step in the test set
for i in range(len(test)):
# fit model and make forecast for history
yhat = model_predict(model, history, cfg)
# store forecast in list of predictions
predictions.append(yhat)
# add actual observation to history for the next loop
history.append(test[i])
# estimate prediction error
error = measure_rmse(test, predictions)
print(' > %.3f' % error)
return error